# More SQL DML and DDL

## Data and environment setup

In [ ]:
!wget http://files.software-carpentry.org/survey.db

In [ ]:
!pip install ipython-sql

In [1]:
%load_ext sql

/usr/local/var/pyenv/versions/miniconda3-latest/envs/mc3-2016-class/lib/python3.5/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
/usr/local/var/pyenv/versions/miniconda3-latest/envs/mc3-2016-class/lib/python3.5/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


In [ ]:
%sql sqlite:///survey.db

## SQL JOINs

Note first that a simple JOIN without further specification of common attributes will result in a cross product relation.  We see this by examining two tables and then taking a simple JOIN.

In [ ]:
%%sql
SELECT *
FROM Site;

In [ ]:
%%sql
SELECT *
FROM Visited;

Now we add the join, and we see the Cartesian product of both tables.  All possible combinations are produced in the resulting relation.

In [ ]:
%%sql
SELECT * 
FROM Site 
JOIN Visited;

Ordinarily we don't want to do this, but it's important to keep in mind that this is how things work under the hood.

More typically, we'll specify at least one pair of common attributes between the two tables to align them properly.

In [ ]:
%%sql
SELECT * 
FROM Site 
JOIN Visited 
  ON Site.name = Visited.site;

Looks much neater, right?

We can of course combine this with naming specific attributes to SELECT (project).

In [ ]:
%%sql
SELECT Site.lat, Site.long, Visited.dated
FROM   Site
JOIN   Visited
ON     Site.name = Visited.site;

There's a shorthand form of the `JOIN` statement that you will often see:  the common attributes can be specified in the `WHERE` clause.

In [ ]:
%%sql
SELECT Site.lat, Site.long, Visited.dated
FROM   Site, Visited
WHERE  Site.name = Visited.site;

Which you use can be a matter of personal preference or style.  For simple queries, this reads very clearly for me.  For more complex queries with more attribute selection conditions in the `WHERE` clause beyond specifying attributes to `JOIN` on, it can be better to split them up.

Compare:

In [ ]:
%%sql
SELECT Site.lat, Site.long, Visited.dated
FROM   Site, Visited
WHERE  Site.name = Visited.site
  AND  Visited.dated IS NOT NULL
  AND  Site.lat < -48
  AND  Site.long > -128;

In [ ]:
%%sql
SELECT Site.lat, Site.long, Visited.dated
FROM   Site
JOIN   Visited
  ON   Site.name = Visited.site
WHERE  Visited.dated IS NOT NULL
  AND  Site.lat < -48
  AND  Site.long > -128;

The above two queries are identical logically, but they read differently.

Finally, we can combine several tables at once - not just two!  This works by adding more tables into the `JOIN` operation.

In [ ]:
%%sql
SELECT Site.lat, Site.long, Visited.dated, Survey.quant, Survey.reading
FROM   Site
JOIN   Visited 
  ON   Site.name = Visited.site
JOIN   Survey 
  ON   Visited.ident = Survey.taken
WHERE  Visited.dated IS NOT NULL;

## Subqueries

We can use the results from one query to constrain conditions on another.  Take for example the Person table:

In [ ]:
%%sql
SELECT DISTINCT ident
FROM person;

This represents a set, which we know we can use with the `IN` clause.

In [ ]:
%%sql
SELECT * 
FROM survey
WHERE person IN ('dyer', 'pb', 'lake', 'row', 'danforth')

But imagine a case where there are dozens, hundreds, or even thousands of possible values.  You want to select carefully, without having to enumerate those possible values.  This is where subqueries work best.

In [ ]:
%%sql
SELECT * 
FROM survey;

In [ ]:
%%sql
SELECT * 
FROM survey
WHERE person IN 
    (SELECT DISTINCT ident
     FROM person);

With this simple approach, you can expand in all directions as you might guess.  For example, let's add further attribute constraint conditions on both the main query and within the subquery.

In [ ]:
%%sql
SELECT * 
FROM survey
WHERE person IN 
    (SELECT DISTINCT ident
     FROM person
     WHERE personal = 'Frank')
AND reading > 7;

This kind of nested subquery is extremely useful and is used quite often.

## Data definition (DDL)

We're going to set up a similar database ourselves, so let's start a new one to avoid messing up the tutorial database.

In [3]:
%sql sqlite:///demo.db

'Connected: None@demo.db'

When creating new tables, we often include a `DROP TABLE` command so that when the code is repeated it performs each `CREATE TABLE` cleanly.  The cleanest way to do this is with `DROP TABLE IF EXISTS`, which typically won't raise an error if the table doesn't already exist.

In [4]:
%%sql
DROP TABLE IF EXISTS Person;
CREATE TABLE Person(
    identity TEXT, 
    personal TEXT, 
    family TEXT);
DROP TABLE IF EXISTS Site;
CREATE TABLE Site(
    name TEXT, 
    lat REAL, 
    long REAL);
DROP TABLE IF EXISTS Visited;
CREATE TABLE Visited(
    ident INTEGER, 
    site TEXT, 
    dated TEXT);
DROP TABLE IF EXISTS Survey;
CREATE TABLE Survey(
    taken INTEGER, 
    person TEXT, 
    quant REAL, 
    reading REAL);

Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

Now we can insert new records into our new tables.

In [5]:
%%sql
INSERT INTO Site values('DR-1', -49.85, -128.57);
INSERT INTO Site values('DR-3', -47.15, -126.72);
INSERT INTO Site values('MSK-4', -48.87, -123.40);
SELECT * FROM Site;

1 rows affected.
1 rows affected.
1 rows affected.
Done.


name,lat,long
DR-1,-49.85,-128.57
DR-3,-47.15,-126.72
MSK-4,-48.87,-123.4


We can do the same thing in a single `INSERT` statement, too.  Note how this time we are specifying our own order of attributes, and our values match that order.  Otherwise, we'd have to follow the schema definition order.

In [6]:
%%sql
DELETE FROM Site;
INSERT INTO Site (lat, long, name) 
VALUES 
    (-49.85, -128.57, 'DR-1'), 
    (-47.15, -126.72, 'DR-3'), 
    (-48.87, -123.40, 'MSK-4')
;
SELECT * FROM Site;

3 rows affected.
3 rows affected.
Done.


name,lat,long
DR-1,-49.85,-128.57
DR-3,-47.15,-126.72
MSK-4,-48.87,-123.4


With `INSERT`, we can also use subqueries.

In [7]:
%%sql
CREATE TABLE JustLatLong(lat text, long text);
INSERT INTO JustLatLong SELECT lat, long FROM Site;
SELECT * FROM JustLatLong;

Done.
3 rows affected.
Done.


lat,long
-49.85,-128.57
-47.15,-126.72
-48.87,-123.4


## More DML:  UPDATE and DELETE

`UPDATE` and `DELETE` statements are powerful as their changes apply immediately across an entire relation.  They are often written with specific constraints that limit their potential effect to precise conditions.

In [8]:
%%sql
SELECT * 
FROM Site
WHERE name = 'MSK-4';

Done.


name,lat,long
MSK-4,-48.87,-123.4


In [9]:
%%sql
UPDATE Site 
SET lat = -48.87, long = -125.40 
WHERE name = 'MSK-4';

1 rows affected.


[]

In [10]:
%%sql
SELECT * 
FROM Site
WHERE name = 'MSK-4';

Done.


name,lat,long
MSK-4,-48.87,-125.4


In [11]:
%%sql
SELECT *
FROM Site;

Done.


name,lat,long
DR-1,-49.85,-128.57
DR-3,-47.15,-126.72
MSK-4,-48.87,-125.4


In [12]:
%%sql
DELETE FROM Site 
WHERE name = 'DR-3';

1 rows affected.


[]

In [13]:
%%sql
SELECT * 
FROM Site;

Done.


name,lat,long
DR-1,-49.85,-128.57
MSK-4,-48.87,-125.4


In [14]:
%%sql
UPDATE Site
SET lat = -50;

2 rows affected.


[]

In [15]:
%%sql
SELECT *
FROM Site;

Done.


name,lat,long
DR-1,-50.0,-128.57
MSK-4,-50.0,-125.4


In [16]:
%%sql
DELETE FROM Site;

2 rows affected.


[]

In [17]:
%%sql
SELECT *
FROM Site;

Done.


name,lat,long
